In [39]:
from searchtweets import load_credentials, gen_request_parameters, collect_results

import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import feature_column
from tensorflow.keras import layers

In [40]:
dataframe = pd.read_csv("./sentimentAnalysis/tweets.csv", usecols=[0, 5], names=["target", "text"])

In [41]:
dataframe = shuffle(dataframe)

In [45]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

def df_to_dataset(dataframe, batch_size):
  dataframe = dataframe.copy()
  texts = dataframe.pop('text')
  labels = dataframe.pop('target')
  return tf.data.Dataset.from_tensor_slices((texts, labels)).batch(batch_size)

batch_size = 320

raw_train_ds = df_to_dataset(train, batch_size)
raw_val_ds = df_to_dataset(val, batch_size)
raw_test_ds = df_to_dataset(test, batch_size)

def normalize_data(input_data):
  lowercase = tf.strings.lower(input_data)
  # todo: filter out usernames
  usernames = tf.strings.regex_replace(lowercase, '@"\B@\w+"', 'user')
  return tf.strings.regex_replace(usernames, '[%s]' % re.escape(string.punctuation), '')

max_features = 10000
sequence_length = 56

vectorize_layer = TextVectorization(
    standardize=normalize_data,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [46]:
model = model.load('../models/sentiment_model')

sentiment_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

examples = [
  "Today is a great day!",
  "This sentence is rather neutral",
  "This show is terrible!"
]

sentiment_model.predict(examples)

NameError: name 'model' is not defined

In [43]:
search_args = load_credentials("./.twitter_keys.yaml", yaml_key="search_tweets_v2")

query = gen_request_parameters(
    "(ether OR eth OR ethereum OR cryptocurrency) -bot -app -is:retweet is:verified lang:en",
    tweet_fields="id,created_at,text",
    results_per_call=100)

tweets = collect_results(query, max_tweets=100, result_stream_args=search_args)

tweet_texts = []

for tweet in tweets:
    if 'text' not in tweet:
        continue
    tweet_text = tweet['text']
    tweet_text = re.sub(pattern="@(\w*)|(\\n)|(https:\/\/t\.co[\w\/]*)", string=tweet_text, repl="")
    tweet_texts.append(tweet_text.lower())

preds = sentiment_model.predict(tweet_texts).flatten()

for i in range(len(tweet_texts)):
    t = tweet_texts[i]
    p = preds[i]
    print("{} - {}".format(t,p))

cannot read file ./.twitter_keys.yaml
Error parsing YAML file; searching for valid environment variables
Your credentials are not configured correctly and  you are missing a required field. Please see the  readme for proper configuration


KeyError: 

In [ ]:
#define moving average function
def moving_avg(x, n):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[n:] - cumsum[:-n]) / float(n)

avg = moving_avg(preds,10)

plt.plot(avg, label="moving average")
plt.plot(preds.flatten(), "o", label="predictions from sentiment analysis")

plt.title('Predictions and MA')
plt.legend(loc='lower right')

plt.show()